<a href="https://colab.research.google.com/github/hyeonsang010716/Machine_Learning/blob/main/%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
review_df = pd.read_csv('./labeledTrainData.tsv' , header = 0 , sep = '\t' , quoting = 3 )
print(review_df.head(3))
print(review_df['review'][0])

         id  sentiment                                             review
0  "5814_8"          1  "With all this stuff going down at the moment ...
1  "2381_9"          1  "\"The Classic War of the Worlds\" by Timothy ...
2  "7759_3"          0  "The film starts with a manager (Nicholas Bell...
"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ

html 태그 및 문자가 아닌 언어 제거

In [2]:
import re
review_df['review'] = review_df['review'].str.replace('<br />' , ' ')
review_df['review'] = review_df['review'].apply(lambda x : re.sub('[^a-zA-Z]' , ' ' , x))
print(review_df['review'][0])

 With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for  

In [6]:
from sklearn.model_selection import train_test_split

class_df = review_df['sentiment']
feature_df = review_df.drop(['id' , 'sentiment'] , axis = 1 , inplace = False)
X_train , X_test , y_train, y_test = train_test_split(feature_df , class_df , test_size = 0.3 , random_state = 156)
X_train.shape , X_test.shape

((17500, 1), (7500, 1))

In [8]:
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score , roc_auc_score

pipeline = Pipeline([
    ('cnt_vect' , CountVectorizer(stop_words = 'english' , ngram_range = (1,2))) ,
    ('lr_clf' , LogisticRegression(C=10))
])
pipeline.fit(X_train['review'] , y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]
print(accuracy_score(y_test ,pred))
print(roc_auc_score(y_test ,pred_probs))


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.886
0.9502702452765763


In [9]:
pipeline = Pipeline([
    ('tfidf_vect' , TfidfVectorizer(stop_words = 'english' , ngram_range = (1,2))) ,
    ('lr_clf' , LogisticRegression(C=10))
])
pipeline.fit(X_train['review'] , y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]
print(accuracy_score(y_test ,pred))
print(roc_auc_score(y_test ,pred_probs))

0.8936
0.959799823582973


In [13]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [14]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

senti_analyzer = SentimentIntensityAnalyzer()
senti_score = senti_analyzer.polarity_scores(review_df['review'][0])
print(senti_score)

{'neg': 0.13, 'neu': 0.743, 'pos': 0.127, 'compound': -0.7943}


In [15]:
def vader_polarity(review , threshold = 0.1):
  analyzer = SentimentIntensityAnalyzer()
  scores = analyzer.polarity_scores(review)
  return 1 if scores['compound'] >= threshold else 0

review_df['vader_preds'] = review_df['review'].apply(lambda x : vader_polarity(x , 0.1))
y_target = review_df['sentiment'].values
vader_preds = review_df['vader_preds'].values
print(accuracy_score(y_target , vader_preds))

0.69556


In [16]:
from sklearn.metrics import accuracy_score , confusion_matrix , precision_score , recall_score
print(confusion_matrix(y_target , vader_preds))
print("정확도 :" , accuracy_score(y_target , vader_preds))
print("정밀도 :" , precision_score(y_target , vader_preds))
print("재현율 :" , recall_score(y_target , vader_preds))


[[ 6747  5753]
 [ 1858 10642]]
정확도 : 0.69556
정밀도 : 0.6491003354681305
재현율 : 0.85136
